untrained as of now

In [7]:
import pandas as pd


df = pd.read_csv(r'C:\Users\mehta\Downloads\qthon\New folder\bftusd.csv')


df.head()


,time,open,close,high,low,volume,Unnamed: 0,target
0,1596710520000,0.000012,0.000012,0.000012,0.000012,1315.994124,0.0,0
1,1596710820000,0.000012,0.000012,0.000012,0.000012,987.170532,1.0,1
2,1596710940000,0.000012,0.000012,0.000012,0.000012,200.000000,2.0,0
3,1596711000000,0.000012,0.000012,0.000012,0.000012,300.000000,3.0,1
4,1596712740000,0.000012,0.000012,0.000012,0.000012,413.121112,4.0,1


deep q learning

In [6]:
import gym
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam
from collections import deque
from random import sample

df = pd.read_csv('C:\\Users\\mehta\\Downloads\\qthon\\New folder\\adabtc.csv')
df['target'] = np.where(df['close'].shift(-1) > df['close'], 1, 0)

class TradingEnvironment(gym.Env):
    def __init__(self, df):
        super(TradingEnvironment, self).__init__()
        self.df = df
        self.current_step = 0
        self.max_steps = len(df) - 1
        self.action_space = gym.spaces.Discrete(3)  # 0: Hold, 1: Buy, 2: Sell
        self.observation_space = gym.spaces.Box(low=0, high=1, shape=(len(df.columns),), dtype=np.float32)

    def reset(self):
        self.current_step = 0
        return self.df.iloc[self.current_step].values

    def step(self, action):
        self.current_step += 1
        done = self.current_step == self.max_steps

        reward = 0
        if action == 1:  
            reward = self.df['close'].iloc[self.current_step] - self.df['close'].iloc[self.current_step - 1]
        elif action == 2:  
            reward = self.df['close'].iloc[self.current_step - 1] - self.df['close'].iloc[self.current_step]

        obs = self.df.iloc[self.current_step].values
        return obs, reward, done, {}


class DQNAgent:
    def __init__(self, state_size, action_size):
        self.state_size = state_size
        self.action_size = action_size
        self.memory = deque(maxlen=2000)
        self.gamma = 0.95  
        self.epsilon = 1.0  
        self.epsilon_decay = 0.995
        self.epsilon_min = 0.01
        self.model = self.build_model()

    def build_model(self):
        model = Sequential()
        model.add(Dense(24, input_dim=self.state_size, activation='relu'))
        model.add(Dense(24, activation='relu'))
        model.add(Dense(self.action_size, activation='linear'))
        model.compile(loss='mse', optimizer=Adam(learning_rate=0.001))
        return model

    def act(self, state):
        if np.random.rand() <= self.epsilon:
            return np.random.choice(self.action_size)
        act_values = self.model.predict(state)
        return np.argmax(act_values[0])

    def remember(self, state, action, reward, next_state, done):
        self.memory.append((state, action, reward, next_state, done))

    def replay(self, batch_size):
        minibatch = sample(self.memory, batch_size)
        for state, action, reward, next_state, done in minibatch:
            target = reward
            if not done:
                target = reward + self.gamma * np.amax(self.model.predict(next_state)[0])
            target_f = self.model.predict(state)
            target_f[0][action] = target
            self.model.fit(state, target_f, epochs=1, verbose=0)
        if self.epsilon > self.epsilon_min:
            self.epsilon *= self.epsilon_decay


env = TradingEnvironment(df)
state_size = env.observation_space.shape[0]
action_size = env.action_space.n
agent = DQNAgent(state_size, action_size)


batch_size = 32
num_episodes = 100
accuracy_list = []

for episode in range(num_episodes):
    state = env.reset()
    state = np.reshape(state, [1, state_size])
    total_reward = 0
    correct_predictions = 0
    total_predictions = 0

    for time in range(env.max_steps):
        action = agent.act(state)
        next_state, reward, done, _ = env.step(action)
        next_state = np.reshape(next_state, [1, state_size])
        agent.remember(state, action, reward, next_state, done)
        state = next_state
        total_reward += reward

        if action == df['target'].iloc[env.current_step - 1]:
            correct_predictions += 1
        total_predictions += 1

        if done:
            accuracy = correct_predictions / total_predictions
            accuracy_list.append(accuracy)
            print(f"Episode: {episode + 1}/{num_episodes}, Total Reward: {total_reward}, Accuracy: {accuracy}")
            break

    if len(agent.memory) > batch_size:
        agent.replay(batch_size)


plt.plot(accuracy_list)
plt.title('Accuracy during Training')
plt.xlabel('Episode')
plt.ylabel('Accuracy')
plt.show()


state = env.reset()
state = np.reshape(state, [1, state_size])
predicted_values = []

for time in range(env.max_steps):
    action = agent.act(state)
    predicted_values.append(action)
    next_state, _, done, _ = env.step(action)
    state = np.reshape(next_state, [1, state_size])


plt.plot(df['target'], label='Actual Values', color='blue')
plt.plot(predicted_values, label='Predicted Values', linestyle='--', color='orange')
plt.title('Actual vs Predicted Values')
plt.xlabel('Time')
plt.ylabel('Action (0: Hold, 1: Buy, 2: Sell)')
plt.legend()
plt.show()


KeyboardInterrupt: 

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report


data = pd.read_csv(r'C:\Users\mehta\Downloads\qthon\New folder\bftusd.csv')


features = data[['open', 'close', 'high', 'low']]


data['trading_signal'] = 'Hold'
data.loc[data['close'] > data['close'].shift(1), 'trading_signal'] = 'Buy'
data.loc[data['close'] < data['close'].shift(1), 'trading_signal'] = 'Sell'


labels = data['trading_signal']


X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=0.2, random_state=42)


X_train = X_train.dropna()
X_test = X_test.dropna()


svm_model = SVC()
svm_model.fit(X_train, y_train)


predictions = svm_model.predict(X_test)


accuracy = accuracy_score(y_test, predictions)
print("Accuracy:", accuracy)
print("Classification Report:\n", classification_report(y_test, predictions))


plt.plot(data['time'], data['close'], label='Close Price')


buy_signals = data[data['trading_signal'] == 'Buy']
plt.scatter(buy_signals['time'], buy_signals['close'], marker='^', color='g', label='Buy Signal')


sell_signals = data[data['trading_signal'] == 'Sell']
plt.scatter(sell_signals['time'], sell_signals['close'], marker='v', color='r', label='Sell Signal')

plt.legend()
plt.show()


KeyboardInterrupt: 